In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
GEB = pd.read_csv("/content/gdrive/My Drive/Data/TPM data list/Capstone Design Data_tpm(5FU).csv",index_col= 0)
GEB_IC = GEB.iloc[37262:37267,:]
GEB_ = GEB.iloc[:37262,:]
real_index = GEB_.index
G_col = GEB_.columns
import numpy as np
log = np.log(GEB_IC.iloc[3,:].astype("float"))         # Target 값인 IC50 Value를 얻어내는 과정
log = log.T
log = pd.DataFrame(log)
log['target'] = log['IC50'].apply(lambda x : 1
                                  if (x > 5)
                                  else 0)
log = log.astype('object')
log = log.drop(['IC50'], axis = 1)
log = log.T

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252

# Data Augumentaion

In [ ]:
GEB_ = GEB_.astype('float')
l = []
v = GEB_.values
for i in range(len(v)):
  for j in range(len(v[i])):
    if v[i][j] != 0:
      l.append(v[i][j])

In [ ]:
mean = np.mean(l)
std = np.std(l)
def n(x):
  if x != 0:
    x = (x-mean)/std  + 0.1
  return x

In [ ]:
for i in GEB_.columns:
  GEB_[i] = GEB_[i].apply(lambda x : n(x))

# Data Preparing

In [ ]:
import numpy as np
dataset = np.load('/content/gdrive/My Drive/csy/dataset_KEGG.npz', allow_pickle=True)
adj_list = dataset['adj_list']   # 각 pathway 별 인접행렬
b_list = dataset['b_list']       # 각 pathway에 속해 있는 gene list
adj_list = adj_list
proteins_by_pathway = b_list

In [ ]:
unique = np.unique(sum(proteins_by_pathway, []))             
gene_index = dict(zip(unique, np.zeros(len(unique))))
for i in b_list:
  for j in i:
    gene_index[j] += 1
import collections
sorted_by_value = sorted(gene_index.items(), key=lambda x : x[1], reverse=True)
sorted_dict = collections.OrderedDict(sorted_by_value)
#print(sorted_dict)
gene_index = list(sorted_dict.keys())
new_gene_indexing = []
for ind, k in enumerate(gene_index):
  if ((ind+1)%2==1):
    new_gene_indexing.insert(0, k)
  else:
    new_gene_indexing.append(k)


# GCN

In [ ]:
import torch
import torch.nn as nn
import torch
import torch.nn as nn
import torch.nn.functional as F
import copy
import scipy.sparse as sp
def clones( module, N):
    return nn.ModuleList(copy.deepcopy(module) for _ in range(N))

class GCNLayer(nn.Module):                                                  # 가장 기본적인 GCN Layer 생성
    
    def __init__(self, in_dim, out_dim, bn=True, num_head=1):                          
        super(GCNLayer, self).__init__()
        self.use_bn = bn                                                    # layer를 통과할 때 마다 포함된 값들을 normalization
        self.linear = nn.Linear(in_dim, out_dim)                            # 각 노드의 feature에 weight와 bias 부여 
        #nn.init.xavier_uniform_(self.linear.weight)
        nn.init.kaiming_normal_(self.linear.weight)
        #self.bn = nn.BatchNorm1d(out_dim)
        self.attention = Attention(out_dim, out_dim, num_head)
    
        
        
    def forward(self, x, adj):
        out = self.linear(x)
        #out = torch.matmul(adj, out)
        out = self.attention(out,adj)
        #out = self.bn(out)
      
        #out = norm(out)                                                  # Activation function, 음수 값은 0으로 내보내는 Relu가 아닌 음수일 때 그 값을 exponenetial 값을 취해주는
        out = F.relu(out)                                                # Elu 함수를 사용
        #out = F.elu(out,alpha=1.0) 
        return out, adj                                                              

class SkipConnection(nn.Module):                                            # ResNet처럼 GCN Block이 통과할 때마다 이전 값을 포함해주는 skip connection 기능을 수행
    
    def __init__(self, in_dim, out_dim):
        super(SkipConnection, self).__init__()
        
        self.in_dim = in_dim
        self.out_dim = out_dim
        
        self.linear = nn.Linear(in_dim, out_dim, bias=False)
        
    def forward(self, in_x, out_x):
        if (self.in_dim != self.out_dim):
            in_x = self.linear(in_x)
        out = in_x + out_x                                                  # GCNBlock class 안에서는 in_x : residual(전 block에서 생성된 output), output : 현재 Block에서 생성된 output 
        return out

class GatedSkipConnection(nn.Module):                                       # residual 학습을 할 때 z_coefficient를 추가하여 residual값과 해당 block output 값의 중요도를 계산하여 추가
    
    def __init__(self, in_dim, out_dim):
        super(GatedSkipConnection, self).__init__()
        
        self.in_dim = in_dim
        self.out_dim = out_dim
        
        self.linear = nn.Linear(in_dim, out_dim, bias=False)
        self.linear_coef_in = nn.Linear(out_dim, out_dim)
        self.linear_coef_out = nn.Linear(out_dim, out_dim)
        nn.init.xavier_uniform_(self.linear.weight)
        nn.init.xavier_uniform_(self.linear_coef_in.weight)
        nn.init.xavier_uniform_(self.linear_coef_out.weight)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, in_x, out_x):
        if (self.in_dim != self.out_dim):
            in_x = self.linear(in_x)
        z = self.gate_coefficient(in_x, out_x)
        out = torch.mul(z, out_x) + torch.mul(1.0-z, in_x)
        return out
            
    def gate_coefficient(self, in_x, out_x):
        x1 = self.linear_coef_in(in_x)
        x2 = self.linear_coef_out(out_x)
        return self.sigmoid(x1+x2)

class GCNBlock(nn.Module):                                                 # 앞서 언급되었던 GCN Layer와 skip connection 기능을 모두 포함하고 있는 GCN Block 생성
    
    def __init__(self, n_layer, in_dim, hidden_dim, out_dim, sc, bn=True): 
        super(GCNBlock, self).__init__()
        
        self.layers = nn.ModuleList()
        for i in range(n_layer):
            self.layers.append(GCNLayer(in_dim if i==0 else hidden_dim,            # n_layer 만큼 GCN layer를 생성하여 nn.MouduleList()에 추가
                                        out_dim if i==n_layer-1 else hidden_dim,
                                        bn=True))
        self.relu = nn.ReLU()
        if sc=='gsc':
            self.sc = GatedSkipConnection(in_dim, out_dim)
        elif sc=='sc':
            self.sc = SkipConnection(in_dim, out_dim)
        elif sc=='no':
            self.sc = None
        else:
            assert False, "Wrong sc type."
        
    def forward(self, x, adj):
        residual = x                                                      
        for i, layer in enumerate(self.layers):
            out, adj = layer((x if i==0 else out), adj)
        if self.sc != None:
            out = self.sc(residual, out)
        out = F.relu(out)
        return out, adj
class Attention(nn.Module):
    
    def __init__(self, in_dim, output_dim, num_head):
        super(Attention, self).__init__()
        
        self.num_head = num_head # multi-head attention: num_head>1
        self.atn_dim = output_dim // num_head
        
        self.linears = nn.ModuleList()
        self.corelations = nn.ParameterList()
        for i in range(self.num_head):
            self.linears.append(nn.Linear(in_dim, self.atn_dim))       # H*W를 만들어주는 과정 [gene X self.atn_dim]
            corelation = torch.FloatTensor(self.atn_dim, self.atn_dim) # attention_dim*attention_dim matrix(C)
            nn.init.xavier_uniform_(corelation)
            self.corelations.append(nn.Parameter(corelation))
      
        self.tanh = nn.Tanh()
        
    def forward(self, x, adj):
        heads = list()
        for i in range(self.num_head):
            x_transformed = self.linears[i](x) # num_aton * attention_dim     
            alpha = self.attention_matrix(x_transformed, self.corelations[i], adj)
            x_head = torch.matmul(alpha, x_transformed)
            heads.append(x_head)
        output = torch.cat(heads, dim=2)
        
        return output
            
    def attention_matrix(self, x_transformed, corelation, adj):
        x = torch.einsum('akj,ij->aki', (x_transformed, corelation))
        alpha = torch.matmul(x, torch.transpose(x_transformed, 1, 2))
        alpha = torch.mul(alpha, adj)
        alpha = self.tanh(alpha)
        return alpha

class ReadOut(nn.Module):                                                # GCN Block을 거쳐 나온 matrix 값을 Nx1 크기로 Flatten (size는 해당 pathway에 속해있는 gene의 개수)
    
    def __init__(self, in_dim, out_dim, act=None):
        super(ReadOut, self).__init__()
        
        self.in_dim = in_dim
        self.out_dim= out_dim
        
        self.linear = nn.Linear(self.in_dim, 
                                self.out_dim)
        #nn.init.xavier_uniform_(self.linear.weight)
        nn.init.kaiming_normal_(self.linear.weight)
        self.activation = act
        #self.norm = nn.LayerNorm(out_dim, elementwise_affine=False)

    def forward(self, x):
        out = self.linear(x)
        out = torch.sum(out, 2)
        if self.activation != None:
            out = self.activation(out)
        #norm = nn.LayerNorm(out.shape[1], elementwise_affine=False)
        #out = norm(out)
        return out



In [ ]:
def mean_norm(df_input):
    return df_input.apply(lambda x: (x-x.mean())/ x.std(), axis=0)
def cell_line_dicimal_norm(G):
   GEB1 = GEB_[[G]]
   GEB1 = GEB1.astype('float')
   GEB1 = mean_norm(GEB1)
   protein_features_df = GEB1.T
   protein_features_df = protein_features_df.astype('float64')
   return protein_features_df
def cell_line_dicimal(G):
   GEB1 = GEB_[[G]]
   GEB1 = GEB1.astype('float')
   #GEB1 = GEB1/292291
   GEB1 = GEB1/479
   protein_features_df = GEB1.T
   protein_features_df = protein_features_df.astype('float64')
   return protein_features_df
#################################
def cell_line_binary(G):                                         # gene expression 값이 Float 형태였던 데이터형태를 이진수로 변환해주는 함수
        GEB1 = GEB_[[G]]
        GEB1['B'] = GEB1[G].astype('str').str.zfill(24)
        non_dot_list = []
        for i in GEB1['B'].values:
          i = i.replace(".", "0")
          non_dot_list.append(i)
        GEB1['B'] = non_dot_list
        GEB2 = GEB1[['B']]
        dat = [list(map(int, str(x).zfill(6))) for x in GEB2.B]
        d = pd.DataFrame(dat, GEB2.index).rename(columns=lambda x: f'2^{23 - x}')
        GEB2.join(d)

        protein_features_df = GEB2.join(d).drop(['B'], axis = 1)
        protein_features_df = protein_features_df.transpose()

        return protein_features_df
##################################        
def selection_(pathway_gene):                                    # 최종적으로 만들어진 GCN output의 크기가 6000x1로 너무 커서 모든 pathway에 값이 0으로 되어 있는 gene을 제거하여  
                                                                # 400~500 x 1 형태로 변환       
    row_list = []
    pathway_gene_ = pathway_gene[0,:,:]
    for row in range(pathway_gene_.shape[1]):
      if (sum(pathway_gene_[:,row] == 0)) < 275 :
        row_list.append(row)
    return pathway_gene[:,:,row_list]

##################################
def normalize_adj(adj):
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1)) # D
    d_inv_sqrt = np.power(rowsum, -0.5).flatten() # D^-0.5
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt) # D^-0.5
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).toarray()

class Attentionisallyouneed(nn.Module):
    def __init__(self):
        super(Attentionisallyouneed, self).__init__()     
        
        self.GCN_layer = nn.ModuleList()                                # GCN Block과 readout을 포함하고 있는 nn.ModuleList() 생성
        self.GCN_layer.append(GCNBlock(1, 1, 2, 2, sc='gsc'))       # nn.Mouduleist()에 포함되어 있는 class 들은 for문을 통해 그 안에 속해있는 모든 function들을 수행
        self.GCN_layer.append(GCNBlock(1, 2, 2, 2, sc='gsc'))       # https://pytorch.org/docs/stable/generated/torch.nn.ModuleList.html?highlight=nn%20modulelist#torch.nn.ModuleList.append
        self.GCN_layer.append(GCNBlock(2, 2, 4, 4, sc='gsc')) 
        self.GCN_layer.append(ReadOut(4, 1))
        self.GCN_layer_list = nn.ModuleList()  

        for _ in list(range(308)):
          self.GCN_layer_list.append(self.GCN_layer)

        self.linear1 = nn.Linear(in_features=25116, out_features=2**7)    # gene은 총 450개
        #self.linear2 = nn.Linear(in_features=2**14, out_features=2**10)
        #self.linear2 = nn.Linear(in_features=2**10, out_features=2**7)
        self.linear3 = nn.Linear(in_features=2**7, out_features=8)
        self.linear4 = nn.Linear(in_features=8, out_features=2)
        self.bn1 = nn.BatchNorm1d(2**7)
        #self.bn2 = nn.BatchNorm1d(8)
        self.bn3 = nn.BatchNorm1d(8)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)

        self.linear_list = nn.ModuleList()
        self.linear_list.append(self.linear1)
        self.linear_list.append(self.bn1)
        self.linear_list.append(self.relu)
        self.linear_list.append(self.dropout)
        self.linear_list.append(self.linear3)
        self.linear_list.append(self.bn3)
        self.linear_list.append(self.relu)
        self.linear_list.append(self.dropout)
        #self.linear_list.append(self.linear3)
        #self.linear_list.append(self.bn3)
        #self.linear_list.append(self.relu)
        self.linear_list.append(self.linear4)
        #self.linear_list.append(self.relu)
        #self.linear_list.append(self.linear5)
        torch.nn.init.kaiming_normal_(self.linear1.weight)
        #torch.nn.init.kaiming_normal_(self.linear2.weight)
        torch.nn.init.kaiming_normal_(self.linear3.weight)
        torch.nn.init.kaiming_normal_(self.linear4.weight)
        
        #self.theta = nn.Parameter(torch.random.normal(0,1,(334)))


    def forward(self, bat_tensor):                      # randome_train_list : 800개의 cell line 중 미리 random 함수를 통해 700개의 cell line list 생성                                                                                   # 메모리 문제로 batch_size를 호출 받아 직접 쌓는 방식으로 구현            
              # 특이하게 forward() 과정에서 batch를 생성 -> [H, W]의 matrix를 직접 쌓아
        batch_size = bat_tensor.shape[0]
        pre_input_tensor = []                                              # [batch, 1, H, W] 형태로 최종 CNN input 텐서 생성              
        pathway_gene = []
        protein_features_df = GEB_.T
        uni_list_prot = np.unique(sum(proteins_by_pathway, []))                  
        indexing = dict(zip(new_gene_indexing, np.arange(len(uni_list_prot))))
        i_ = list(range((342)))                                                    
        col_list = list(protein_features_df.columns)
        pathway_gene = []

        for i, adj, b in zip(i_, adj_list, b_list):                         # Pathway 별로 GCN 연산 수         
          ##
          col_gene_ind = []
          for col_gene in b:
            col_gene_ind.append(col_list.index(col_gene))
          ##

          x = bat_tensor[:,col_gene_ind,:]
          adj = normalize_adj(adj)         
          x, adj = torch.tensor(x).to('cuda').float(), torch.tensor(adj).to('cuda').float()    # x : feature matrix   adj : 각 pathway의 인접행렬
          #adj = torch.tensor(adj).to('cuda').float() 
          #print(x.shape, adj.shape)  
          
            # GCN 학습 과정
        
          for k, layer in enumerate(self.GCN_layer_list[i]): 
            if k != 3:                                                    # -> GCNBlock(GCNLayer 10개) -> GCNBlock(GCNLayer 25개) -> Readout -> gene의 숫자가 size인 텐서 생성 
              
              out, adj = layer((x if k==0 else out), adj)             
        
            else:              
              output = layer(out) 
          
          if i == 0:
            pathway_gene = output
          else:
            #pathway_gene = torch.cat([pathway_gene, output*self.theta[i]], dim=1)         # loss = crieter ~~ + torch.mean(torch.abs(model.theta))
            pathway_gene = torch.cat([pathway_gene, output], dim=1)
        input_data_depth10 = pathway_gene
        x = input_data_depth10.view(batch_size, -1).to('cuda')
        for layer in self.linear_list:
          x = layer(x)
        t = 0
        
        return x, t


# torch.Size([1, 308, 351])


In [ ]:
def make_data(GEB, train_set):
  random_cell_line = []       
  bat_tensor = []  
  for cnt, G in enumerate(GEB.columns[list(train_set)]):   
    random_cell_line.append(G)
    #protein_features_df = cell_line_dicimal_norm(G)
    values = cell_line_dicimal(G).T.values
    tensor = torch.from_numpy(values)
    bat_tensor.append(tensor)
  bat_tensor = torch.stack(bat_tensor, dim = 0)
  target = torch.tensor(pd.Series(log[random_cell_line].values[0])).to('cuda')
  return bat_tensor, target  
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp  

def eval(model, eval_list, batch_size):
  model.eval()
  correct = 0
  loss = 0
  with torch.no_grad():
    eval_set = np.random.choice(eval_list, batch_size)
    bat_tensor, target = make_data(GEB, eval_set) 
    output, t = model(bat_tensor)  
    loss = criterion(output, target)
    auc_score = roc_auc_score(target.cpu().numpy(), output.cpu().detach().numpy()[:,1]) 
    _, predicted = output.max(1)
    correct = predicted.eq(target).sum().item()
  return 100. * correct / batch_size, loss.item(), auc_score

import random
random_seed = 1553
torch.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
#np.random.seed(random_seed)
random.seed(random_seed)
#torch.cuda.manual_seed(random_seed)
#torch.cuda.manual_seed_all(random_seed) # multi-GPU
device = 'cuda'

------------------여기까지 실행------------------------------

EPOCH까지 연산

# Training

In [ ]:
from sklearn.metrics import roc_auc_score
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

import warnings
warnings.filterwarnings(action='ignore')

import random
random_seed = 1553
torch.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
#np.random.seed(random_seed)
random.seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.cuda.manual_seed_all(random_seed) # multi-GPU
#####################################################


#nums = set()
#while len(nums) != 800: 
#  nums.add(random.randint(1, 7210))
#eval_list = list(nums)
#train_list = list(range(1, 7210))
#for i in eval_list:
#  train_list.remove(i)

nums = set()
while len(nums) != 100: 
  nums.add(random.randint(1, 801))
eval_list = list(nums)
train_list = list(range(1, 801))
for i in eval_list:
  train_list.remove(i)
######################################################
 # hidden_dim, de_hidden_dim, n_layers, n_heads, pf_dim, dropout_ratio, device
device = 'cuda'

model = Attentionisallyouneed().to(device)  # 메모리 때문에 308 -> 89
#model = GraphMLP().to(device)  

#optimizer = torch.optim.Adam(model.parameters(), lr=5e-4) # lr = 1e-3 or 5e-4, weight_decay=0.001
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4, weight_decay = 0.01)#, weight_decay=0.1
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10, eta_min=0,  verbose = False)
#scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
criterion = nn.CrossEntropyLoss()
#model.train()  


print("total parameter 수 : ", get_n_params(model))
# 275664581
# 423843235 -> gene 137
# 544179655 -> gene 176
loss_list = []
val_loss_list = []
import time
start = time.time()
batch_size = 64
iterations = 10
epochs = 15

for epoch in range(epochs):
  #model.train() 
  eval_acc_sum = 0
  epoch_loss = 0
  epoch_val_loss = 0
  train_list = list(range(1, 801))
  #train_list = []
  for i in eval_list:
    train_list.remove(i)
  
  for iteration in range(iterations): 
    train_set = np.random.choice(train_list, batch_size, replace = False)
    for t in train_set:
      train_list.remove(t) 

    bat_tensor, target = make_data(GEB, train_set)   
    model.train()
    train_loss = 0 
    optimizer.zero_grad()
    output, t = model(bat_tensor) # 최종 output
   
    loss = criterion(output, target)
    train_loss += loss.item()
    loss.backward()  # 역전파
    optimizer.step()
    #scheduler.step()
    torch.cuda.empty_cache()
    print()
    print(iteration, "번째 iteration")
    print("train loss : ", train_loss)        
    #auc_score = roc_auc_score(target.cpu().numpy(), output.cpu().detach().numpy()[:,1]) 
    if iteration % 1 == 0:

      eval_acc, eval_loss, auc = eval(model, eval_list, 64)
      print("\n eval loss : ", eval_loss,)
      print("eval accuarcy : ", eval_acc, "%")
      print('AUC : ', auc  )
      print('------------------------------------\n')
      eval_acc_sum += eval_acc
      val_loss_list.append(eval_loss)
      epoch_val_loss += eval_loss
      if auc > 0.81:
        break

    loss_list.append(train_loss)
    epoch_loss += train_loss
    #print(model.linear1.weight.grad)
    #print(model.GCN_layer[3].linear.weight.grad)
  print("------------------{} EPOCH 완료-----------------------".format(epoch))
  print("Epoch Loss Mean : ", epoch_loss/10) # batch 64 -> 10
  print("Epoch Val Loss Mean : ", epoch_val_loss/10)
  print("Epoch eval acc : ", eval_acc_sum/10)
  print("---------------------------------------------------")
  
print("time :", (time.time() - start)/60)

# weight decay : 0,1 ~  0.000001 까지 다양하게 해보기
# adamW 사용해보기

total parameter 수 :  3216515

0 번째 iteration
train loss :  0.7055772542953491

 eval loss :  7.2107648849487305
eval accuarcy :  34.375 %
AUC :  0.5968614718614719
------------------------------------


1 번째 iteration
train loss :  0.6954653859138489

 eval loss :  5.377948760986328
eval accuarcy :  43.75 %
AUC :  0.5376984126984127
------------------------------------


2 번째 iteration
train loss :  0.775611162185669

 eval loss :  4.638859748840332
eval accuarcy :  57.8125 %
AUC :  0.6286286286286286
------------------------------------


3 번째 iteration
train loss :  0.6830260157585144

 eval loss :  8.118531227111816
eval accuarcy :  48.4375 %
AUC :  0.6207233626588465
------------------------------------


4 번째 iteration
train loss :  0.739031195640564

 eval loss :  10.36352252960205
eval accuarcy :  46.875 %
AUC :  0.46470588235294114
------------------------------------


5 번째 iteration
train loss :  0.6897194385528564

 eval loss :  16.696269989013672
eval accuarcy :  40.625 %
A

In [ ]:
torch.sum(weight,0).size()

torch.Size([25116])

In [ ]:
weight = model.linear1.weight
weight_sum = torch.max(weight,0)
weight_sum

torch.return_types.max(values=tensor([0.0317, 0.0266, 0.0190,  ..., 0.0205, 0.0212, 0.0178], device='cuda:0',
       grad_fn=<MaxBackward0>), indices=tensor([ 77,  32, 101,  ..., 103, 120, 101], device='cuda:0'))

In [ ]:
weight = model.linear1.weight
weight_sum = torch.sum(weight,0)
weight_sum = weight_sum.cpu().detach().numpy()
sort_index = pd.DataFrame(weight_sum).sort_values(0, ascending=False).head(20).index
sort_index

Int64Index([20865, 24045,  9754,  3187, 19928, 18297,  2771,  6727, 23035,
            19037, 15742, 21700,  1626,  4506, 13636, 15460,  6721, 17027,
            10522,  6520],
           dtype='int64')

In [ ]:
len_list = []
for i in range(len(b_list)):
  len_list.append(len(b_list[i]))

In [ ]:
for number in sort_index:
  index_sum = 0
  f = open("/content/gdrive/My Drive/Data/hsa_order.txt")
  for i in range(len(len_list)):
    #number = 19928
    if index_sum <= number < (index_sum+len_list[i]):
      #print(i)
      print(f.readlines()[i])
      print(b_list[i][number - index_sum])
      print()
      print()
      break
    index_sum += len_list[i]

hsa05164

NOTCH1


hsa05218

RAD51


hsa04371

PRKAB1


hsa05410

HPRT1


hsa05142

PLEKHM2


hsa04978

PSMC1


hsa05332

AOX1


hsa04115

CCNB3


hsa05200

GSTP1


hsa05100

NCKAP1


hsa04922

PRKAG3


hsa05160

IL15RA


hsa00532

GANC


hsa03440

PPP3CA


hsa04722

CACNA1S


hsa04914

ADCY6


hsa04115

MCM4


hsa04964

CHRM3


hsa04514

ICOSLG


hsa04141

MAP2K2




In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.lineplot(x=np.arange(len(val_loss_list)), y = val_loss_list)
#sns.lineplot(x=np.arange(len(loss_list)), y = loss_list)
#plt.legend(labels=["val_loss","train_loss"])

In [ ]:
sns.lineplot(x=np.arange(len(loss_list)), y = loss_list)